In [ ]:
# Databricks notebook: Download artifacts for latest run of a given task of THIS job
# ---------------------------------------------------------------------------------

# PARAMETERS
# ----------
dbutils.widgets.text("TASK_KEY", "")               # task_key (task name) within this job
dbutils.widgets.text("SOURCE_PATH", "")            # subfolder inside artifacts, empty for root
dbutils.widgets.text("DEST_PATH", "")   # local/DBFS dest on this cluster

In [ ]:
TASK_KEY = dbutils.widgets.get("TASK_KEY").strip()
SOURCE_PATH = dbutils.widgets.get("SOURCE_PATH").strip()
DEST_PATH = dbutils.widgets.get("DEST_PATH").strip()

In [ ]:
import subprocess

In [ ]:
def run_cmd(cmd: str) -> str:
    """Run a shell command and return stdout; raise on error."""
    print(f"Executing: {cmd}")
    result = subprocess.run(
        cmd,
        shell=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
    )
    if result.returncode != 0:
        raise RuntimeError(
            f"Command failed ({result.returncode}).\n"
            f"STDOUT:\n{result.stdout}\n\nSTDERR:\n{result.stderr}"
        )
    return result.stdout

In [ ]:
# 1. Configure CLI to use Jobs API 2.1
out_config = run_cmd("databricks jobs configure --version=2.1")
print(out_config)

In [ ]:
# 1. Get JOB_ID of *this* notebook's job
# --------------------------------------
JOB_ID = dbutils.notebook().getContext().toJson()['job_id']
JOB_ID = dbutils.notebook.entry_point.getDbutils().notebook().getContext().toJson() 
print(f"Current job id (JOB_ID) = {JOB_ID}")

In [ ]:
# Import Databricks SDK
from databricks.sdk import WorkspaceClient
import json

In [ ]:
# Create client using the job's identity (service principal / compute identity)
w = WorkspaceClient()

# Get run info (Jobs API 2.1)
run = w.jobs.get_run(run_id=int(JOB_ID))

# `run` is a JobsGetRunResponse; `run.tasks` is a list of RunTask instances
tasks = run.tasks or []

# Pretty-print equivalent to `jq .tasks`
print(json.dumps([t.as_dict() for t in tasks], indent=2))